# 章節 11：假設檢定（Hypothesis Testing）

## 11.2 公平的比較

為了要做到公平的比較，我們要在不看資料的情況下定義假設 B 。所以我們嘗試不同的定義。

如果你檢查歐元硬幣，你可能注意到「正面」比「背面」更為凸出。你可能認為這個形狀會影響 x （出現正面的機率），但又不確定會出現比較多或比較少正面。

所以你可能說「我認為硬幣是有偏差的，所以 x 可能是 0.6 或 0.4，但我不確定是哪種」。這個版本的假設稱作 b_two，其中包含兩個子假設。我們對每個子假設計算其似然性，然後平均。

In [6]:
def Likelihood(data, hypo):
    x = hypo / 100.0
    heads, tails = data
    like = x**heads * (1-x)**tails
    return like

# 正面 140 次，反面 110 次
data = (140, 110)

like40 = Likelihood(data, 40)
like60 = Likelihood(data, 60)

b_two_likelihood = 0.5 * like40 + 0.5 * like60
f = Likelihood(data, 50)

print("b_two 與硬幣公平假設 f 的比率：", b_two_likelihood/f)

b_two 與硬幣公平假設 f 的比率： 1.3312060570100888


b_two 的似然性比（貝葉斯因子）為 1.3，這表示資料很微弱的支持假設 b_two。

更普遍的來說，假設你懷疑硬幣是偏差的，但你沒有 x 的線索。

你可以建立一個 Suite 物件稱作 b_uniform 來代表從 0 到 100 的子假設。然後移除 x=50% 的假設(假設硬幣公平)，雖然移除對於結果幾乎沒有影響。

In [33]:
import thinkbayes as  tb

class Euro(tb.Suite):
    
    def Likelihood(self, data, hypo):
        x = hypo / 100.0
        heads, tails = data
        like = x**heads * (1-x)**tails
        return like


def SuiteLikelihood(suite, data):
    total = 0
    for hypo, prob in suite.Items():
        like = suite.Likelihood(data, hypo)
        
        # 計算 b_uniform 的平均似然性
        total += prob * like
    return total

b_uniform = Euro(range(0, 101))
b_uniform.Remove(50)
b_uniform.Normalize()

data = (140, 110)
print("b_uniform 與硬幣公平的比率：", SuiteLikelihood(b_uniform, data)/f)

b_uniform 與硬幣公平的比率： 0.4667233591607601


b_uniform 的然性比率約為 0.47，表示資料微弱的反對 b_uniform 假設，相對於 F 假設。

SuiteLikelihood 方法其實很類似 Update 方法：

<pre>
def Update(self, data):
    for hypo in self.Values():
        like = self.Likelihood(data, hypo)
        self.Mult(hypo, like)
        
    return self.Normalize()


def Normalize(self):
    total = self.Total()
    factor = 1.0 / total

    for x in self.d:
        self.d[x] *= factor
        
    return total
</pre>

Normalize 方法的回傳值就是 Suite 的機率總和，也就是每個子假設的似然性的平均值，權重就是每個先驗的機率。然後 Update 方法會回傳總結果，所以我們可以改用下面方式求 b_uniform 的似然性。最後比率的結果是一樣的。

In [34]:
b_uniform_likelihood = b_uniform.Update(data)
print("b_uniform 與硬幣公平的比率：", b_uniform_likelihood/f)

b_uniform 與硬幣公平的比率： 0.4667233591607601
